In [ ]:
import sys
sys.path.append('..')

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = '1'

In [ ]:
!pip install umap-learn

In [ ]:
import time
from gerumo.data.dataset import describe_dataset
from gerumo.data.generators import build_generator, BaseGenerator
from gerumo.utils.engine import (
    setup_cfg, setup_environment, setup_experiment, setup_model,
    build_dataset, build_callbacks, build_metrics, build_optimizer, build_loss
)
from gerumo.models.base import build_model, BaseModel, load_model
from gerumo.visualization.metrics import training_history


class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
args = dotdict()

In [ ]:
args['config_file'] = '/home/ir-riqu1/rds/rds-iris-ip007/ir-riqu1/outputs/best models/20-04-01_onioncnn_ftt_lst_23_epochs_dallcut1000_lr_0.02_f2_sgd_clr_with_momentum_classification_20220420_183240/config.yml'
args['opts'] = [
    u1/rds/rds-iris-ip007/ir-riqu1/Prod5-parquets/output_tiny_splitted/train_events.parquet',
    'DATASETS.TRAIN.TELESCOPES','/home/ir-riqu1/rds/rds-iris-ip007/ir-riqu1/Prod5-parquets/output_tiny_splitted/train_telescopes.parquet',
    'DATASETS.VALIDATION.EVENTS','/home/ir-riqu1/rds/rds-iris-ip007/ir-riqu1/Prod5-parquets/output_ALL_cut500_splitted/validation_events.parquet',
    'DATASETS.VALIDATION.TELESCOPES','/home/ir-riqu1/rds/rds-iris-ip007/ir-riqu1/Prod5-parquets/output_ALL_cut500_splitted/validation_telescopes.parquet',
    'SOLVER.BATCH_SIZE',128
]

In [ ]:
cfg = setup_cfg(args)
output_dir = setup_experiment(cfg)
logger = setup_environment(cfg)

In [ ]:
train_dataset = build_dataset(cfg, 'train')
describe_dataset(train_dataset, logger,
                save_to=output_dir / "train_description.txt")
validation_dataset = build_dataset(cfg, 'validation')
describe_dataset(validation_dataset, logger,
                save_to=output_dir / "validation_description.txt")

In [ ]:
validation_dataset.particle_type.value_counts(normalize=True)

In [ ]:
train_generator = build_generator(cfg, train_dataset)
validation_generator = build_generator(cfg, validation_dataset)

In [ ]:
train_generator.verbose_mode()
input_shape = train_generator.get_input_shape()
model = build_model(cfg, input_shape)

In [ ]:
output_dir = '/home/ir-riqu1/rds/rds-iris-ip007/ir-riqu1/outputs/best models/20-04-01_onioncnn_ftt_lst_23_epochs_dallcut1000_lr_0.02_f2_sgd_clr_with_momentum_classification_20220420_183240'

In [ ]:
loaded_model=load_model(model,validation_generator,output_dir=output_dir)

In [ ]:
callbacks = build_callbacks(cfg)
metrics = build_metrics(cfg)
optimizer = build_optimizer(cfg)
loss = build_loss(cfg)

In [ ]:
model = setup_model(
    loaded_model, train_generator, optimizer, loss, metrics
)

In [ ]:
train_generator.fit_mode()
validation_generator.fit_mode()
model.fit_mode()

In [ ]:
validation_dataset.groupby('particle_type')['event_unique_id'].count()


In [ ]:
validation_dataset['hillas_intensity'].plot(kind="hist",logx=True)

In [ ]:
model.evaluate(validation_generator)

In [ ]:
import numpy as np
x,y= validation_generator[0]
y_pred=model(x)
y_pred=np.argmax(y_pred, axis = 1)
y_pred=np.array(y_pred).reshape(128,1)
mask=(y==y_pred)
y_bc=~mask*10
y_new=y+y_bc
for i in range(len(y_new)):
    if y_new[i]>2:
        y_new[i]=2

PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
for L in model._logic_blocks:
    pca = PCA(n_components=2)
    scaler = StandardScaler()
    o = L(x)
    scaler.fit(o)
    o=scaler.transform(o)   #descomentar para normalizar
    pca.fit(o)
    o_pca = pca.transform(o)
    o_embedded = pca.transform(o)   # descomentar para hacer SOLO pca
    plt.figure()
    scatter=plt.scatter(o_embedded[:,0],o_embedded[:,1],c=y_new,cmap='brg',label=y_new[:,0], alpha=0.5)
    plt.legend(handles=scatter.legend_elements()[0], labels=['proton','gamma','misclassified'])
    plt.title('PCA layer:%s' %o.shape[1])
    print(o.shape)
    print(pca.explained_variance_ratio_.cumsum()[-1])

TSNE

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

for L in model._logic_blocks:
    pca = PCA(n_components=50)
    tsne = TSNE(n_components=2,perplexity=30, learning_rate='auto', init='random',random_state=42, metric='correlation', method='exact',early_exaggeration=100,square_distances=True)
    scaler = StandardScaler()
    o = L(x)
    scaler.fit(o)
    o=scaler.transform(o)   #descomentar para normalizar
    pca.fit(o)
    o_pca = pca.transform(o)
    o_embedded = tsne.fit_transform(o)
    #o_embedded = pca.transform(o)   # descomentar para hacer SOLO pca
    plt.figure()
    scatter=plt.scatter(o_embedded[:,0],o_embedded[:,1],c=y_new,cmap='brg',label=y_new[:,0], alpha=0.5)
    plt.legend(handles=scatter.legend_elements()[0], labels=['proton','gamma','misclassified'])
    plt.title('PCA TSNE layer:%s' %o.shape[1])
    print(o.shape)
    print(pca.explained_variance_ratio_.cumsum()[-1])

UMAP

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import umap
from sklearn.preprocessing import StandardScaler

for L in model._logic_blocks:
    reducer = umap.UMAP(metric='correlation',random_state=42)
    pca = PCA(n_components=50)
    scaler = StandardScaler()
    o = L(x)
    scaler.fit(o)
    o=scaler.transform(o)   #descomentar para normalizar
    pca.fit(o)
    o_pca = pca.transform(o)
    o_embedded = reducer.fit_transform(o,y)
    #o_embedded = pca.transform(o)   # descomentar para hacer SOLO pca
    plt.figure()
    scatter=plt.scatter(o_embedded[:,0],o_embedded[:,1],c=y_new,cmap='brg',label=y_new[:,0], alpha=0.5)
    plt.legend(handles=scatter.legend_elements()[0], labels=['proton','gamma','misclassified'])
    plt.title('Supervised UMAP layer:%s' %o.shape[1])
    print(o.shape)
    print(pca.explained_variance_ratio_.cumsum()[-1])


UMAP on flatten layer

In [ ]:
reducer = umap.UMAP(n_neighbors=20, min_dist=0.0001,n_components=2, metric='correlation',random_state=42)
pca = PCA(n_components=100)
scaler = StandardScaler()
o=model.encoder(x)
scaler.fit(o)
o=scaler.transform(o)   #descomentar para normalizar
pca.fit(o)
o_pca = pca.transform(o)
o_embedded = reducer.fit_transform(o)
#o_embedded = pca.transform(o)   # descomentar para hacer SOLO pca
plt.figure()
scatter=plt.scatter(o_embedded[:,0],o_embedded[:,1],c=y_new,cmap='brg',label=y_new[:,0], alpha=0.5)
plt.legend(handles=scatter.legend_elements()[0], labels=['proton','gamma','misclassified'])
plt.title('non-supervised UMAP on flatten layer:%s' %o.shape[1])
print(o.shape)
print(pca.explained_variance_ratio_.cumsum()[-1])

Supervised UMAP on flatten layer 

In [ ]:
reducer = umap.UMAP(n_neighbors=25, min_dist=0.00001,n_components=2,metric='correlation',random_state=42)
pca = PCA(n_components=100)
scaler = StandardScaler()
o=model.encoder(x)
scaler.fit(o)
o=scaler.transform(o)   #descomentar para normalizar
pca.fit(o)
o_pca = pca.transform(o)
o_embedded = reducer.fit_transform(o,y)
#o_embedded = pca.transform(o)   # descomentar para hacer SOLO pca
plt.figure()
scatter=plt.scatter(o_embedded[:,0],o_embedded[:,1],c=y_new,cmap='brg',label=y_new[:,0], alpha=0.5)
plt.legend(handles=scatter.legend_elements()[0], labels=['proton','gamma','misclassified'])
plt.title('Supervised UMAP flatten layer:%s' %o.shape[1])
print(o.shape)
print(pca.explained_variance_ratio_.cumsum()[-1])